In [1]:
import os

import geopandas as gpd
import rasterio
from PIL import Image
from rasterio import features

Image.MAX_IMAGE_PIXELS = 999_999_999

---

## Test

In [ ]:
shp_fn = 'data/og1-shp.shp'
rst_fn = 'data/OG1_Orthomosaic_jpegcompression.tif'
out_fn = 'data/rasterized.tif'

In [ ]:
fractures = gpd.read_file(shp_fn)
fractures.head()

In [ ]:
rst = rasterio.open(rst_fn)

meta = rst.meta.copy()
meta.update(compress='lzw')

In [ ]:
with rasterio.open(out_fn, 'w+', **meta) as out:
    out_arr = out.read(1)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(fractures.geometry, fractures.frac*255))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)
    out.write_band(2, burned)
    out.write_band(3, burned)

---

## Production

In [2]:
def rasterize(shp_fn, rst_fn, out_fn):
    fractures = gpd.read_file(shp_fn)
    rst = rasterio.open(rst_fn)

    meta = rst.meta.copy()
    meta.update(compress='lzw')

    with rasterio.open(out_fn, 'w+', **meta) as out:
        out_arr = out.read(1)
    
        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(fractures.geometry, fractures.frac*255))
    
        burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
        out.write_band(1, burned)
        out.write_band(2, burned)
        out.write_band(3, burned)

    print(f"Trace §--- {rst_fn} ---§ rasterized successfully!")

In [14]:
orthos_path = '/Users/afatihi/DATA/loviisa_ovaskainen/Loviisa_orthomosaics_for_automation/tifs'

orthos = [fname for fname in os.listdir(orthos_path) if fname.endswith('tif')]
orthos.sort()

traces_path = '/Users/afatihi/DATA/loviisa_ovaskainen/data-exported-ESRI-Shapefile/traces/zonal'

traces = [fname for fname in os.listdir(traces_path) if fname.endswith('shp')]
traces.sort()

orthos = [i for i in orthos if 'KL2' not in i]
db = []
for i, o_t in enumerate(zip(orthos, traces)):
    if 'KL2' in o_t[0]: continue  # not enough traces !!
    db.append(o_t)

db

[('KB11_orto_3b.tif', '_KB11_tulkinta.shp'),
 ('KB2_orto_3b.tif', '_KB2_tulkinta_clip.shp'),
 ('KB3_orto.tif', '_KB3_tulkinta_Bc_edits_clip.shp'),
 ('KB7_orto.tif', '_KB7_tulkinta.shp'),
 ('KB9_orto.tif', '_KB9_tulkinta_clip1.shp'),
 ('KL5_orto.tif', '_KL5_tulkinta.shp'),
 ('OG1_Orthomosaic_jpegcompression.tif', '_OG1_tulkinta.shp')]

In [4]:
for (ortho, trace) in db:
    out = f"{ortho.split('_')[0]}-mask.tif"
    rasterize(
        os.path.join(traces_path, trace),
        os.path.join(orthos_path, ortho),
        out)

Trace §--- /Users/afatihi/DATA/loviisa_ovaskainen/Loviisa_orthomosaics_for_automation/tifs/KB11_orto.tif ---§ rasterized successfully!
Trace §--- /Users/afatihi/DATA/loviisa_ovaskainen/Loviisa_orthomosaics_for_automation/tifs/KB2_orto.tif ---§ rasterized successfully!
Trace §--- /Users/afatihi/DATA/loviisa_ovaskainen/Loviisa_orthomosaics_for_automation/tifs/KB3_orto.tif ---§ rasterized successfully!
Trace §--- /Users/afatihi/DATA/loviisa_ovaskainen/Loviisa_orthomosaics_for_automation/tifs/KB7_orto.tif ---§ rasterized successfully!
Trace §--- /Users/afatihi/DATA/loviisa_ovaskainen/Loviisa_orthomosaics_for_automation/tifs/KB9_orto.tif ---§ rasterized successfully!
Trace §--- /Users/afatihi/DATA/loviisa_ovaskainen/Loviisa_orthomosaics_for_automation/tifs/KL5_orto.tif ---§ rasterized successfully!


In [20]:
for (ortho, trace) in db:
    out = f"{ortho.split('_')[0]}-mask.tif"
    rasterize(
        os.path.join(traces_path, trace),
        os.path.join(orthos_path, ortho),
        out)

Trace §--- /Users/afatihi/DATA/loviisa_ovaskainen/Loviisa_orthomosaics_for_automation/tifs/KL5_orto.tif ---§ rasterized successfully!
